In [31]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests 
import re

In [3]:
top100_whisky = pd.read_csv('Top100_whisky.csv')
top100_whisky.head()

,Name,Year,hkls_name,hkls_link,price_name,price_link,wastons_name,wastons_link
0,Lagavulin,16,Lagavulin 16 Years Single Malt Scotch Whisky,https://www.hkliquorstore.com/index.php/lagavu...,Lagavulin Aged 16 Years,https://www.price.com.hk/product.php?p=270584,Lagavulin 16 Years Old Single Malt Whisky,https://www.watsonswine.com/en/wine/lagavulin-...
1,Ardbeg,10,Ardbeg Ten Years Old Single Malt Whisky,https://www.hkliquorstore.com/index.php/ardbeg...,Ardbeg 10 Years Old Single Malt Whisky 700ml,https://www.price.com.hk/product.php?p=341421,Ardbeg 10 Year Old Islay Single Malt Whisky,https://www.watsonswine.com/en/wine/ardbeg-10-...
2,Balvenie,12,The Balvenie DoubleWood 12 Years Single Malt W...,https://www.hkliquorstore.com/index.php/the-ba...,Balvenie 12 Year Old DoubleWood,https://www.price.com.hk/product.php?p=220964,Balvenie Doublewood 12 Years Single Malt Scotc...,https://www.watsonswine.com/en/wine/balvenie-d...
3,Laphroaig,10,Laphroaig 10 Years Single Malt Scotch Whisky,https://www.hkliquorstore.com/index.php/laphro...,Laphroaig 10 Years Old,https://www.price.com.hk/product.php?p=256951,Laphroaig 10 Year Old Single Malt Whisky,https://www.watsonswine.com/en/wine/laphroaig-...
4,Glenmorangie,10,Glenmorangie The Original 10 Years Single Malt...,https://www.hkliquorstore.com/index.php/glenmo...,"Glenmorangie 10 YEAR OLD THE ORIGINAL', 'Glenm...",https://www.price.com.hk/product.php?p=417769'...,NaN,NaN


## Create dataframe 

In [25]:
# create whisky details csv for hkls 
hkls_details = top100_whisky.iloc[:,0:4].copy()
hkls_details.head()

# add new columns in hkls_details
hkls_details['Price'] = pd.Series(dtype='str')
hkls_details['Stock'] = pd.Series(dtype='str')
hkls_details['Volume'] = pd.Series(dtype='str')

for w in range(len(hkls_details['Name'])):
    if len(str(hkls_details['hkls_name'][w]))>5:
        URL = hkls_details['hkls_link'][w]
        html = requests.get(URL)
        page = BeautifulSoup(html.text,'html.parser')

        hkls_details['Price'][w] = page.find(class_ = 'price').text
        if page.find (class_ = 'stock available'):
            hkls_details['Stock'][w] = True
        else:
            hkls_details['Stock'][w] = False
        try:
            hkls_details['Volume'][w] = re.search(r'(Alcohol)(\s)(\d+\w+)$', page.find(class_= 'product attribute overview').text)[3]
        except:
            hkls_details['Volume'][w] = 'Not available'

hkls_details.to_csv('hkls_details.csv', index = False)

## Update dataframe 

In [53]:
def update_hkls():
    import pandas as pd
    from bs4 import BeautifulSoup 
    import requests 
    import re

    hkls_details = pd.read_csv('hkls_details.csv')

    # update Price and Stock
    for w in range(len(hkls_details['Name'])):
        if len(str(hkls_details['hkls_name'][w]))>5:
            URL = hkls_details['hkls_link'][w]
            html = requests.get(URL)
            page = BeautifulSoup(html.text,'html.parser')

            hkls_details['Price'][w] = int(re.search(r'\$\d+,\d+|\$\d+',page.find(class_ = 'price').text).group().replace('$', '').replace(',',''))
            if page.find (class_ = 'stock available'):
                hkls_details['Stock'][w] = True
            else:
                hkls_details['Stock'][w] = False
            
    hkls_details.to_csv('hkls_details.csv', index = False)
    return hkls_details


In [54]:
update_hkls()

,Name,Year,hkls_name,hkls_link,Price,Stock,Volume
0,Lagavulin,16,Lagavulin 16 Years Single Malt Scotch Whisky,https://www.hkliquorstore.com/index.php/lagavu...,888,True,700ml
1,Ardbeg,10,Ardbeg Ten Years Old Single Malt Whisky,https://www.hkliquorstore.com/index.php/ardbeg...,648,True,700ml
2,Balvenie,12,The Balvenie DoubleWood 12 Years Single Malt W...,https://www.hkliquorstore.com/index.php/the-ba...,688,True,700ml
3,Laphroaig,10,Laphroaig 10 Years Single Malt Scotch Whisky,https://www.hkliquorstore.com/index.php/laphro...,688,True,700ml
4,Glenmorangie,10,Glenmorangie The Original 10 Years Single Malt...,https://www.hkliquorstore.com/index.php/glenmo...,498,True,700ml
...,...,...,...,...,...,...,...
95,Chivas Regal,18,Chivas Regal 18 Years,https://www.hkliquorstore.com/index.php/chivas...,828,True,700ml
96,Bruichladdich,the classic laddie,Bruichladdich Scottish Barley The Classic Laddie,https://www.hkliquorstore.com/index.php/bruich...,648,True,700ml
97,Glen Moray,15,NaN,NaN,NaN,NaN,NaN
98,Aberfeldy,16,Aberfeldy 16 Years Single Malt Whisky,https://www.hkliquorstore.com/index.php/aberfe...,1278,True,700ml


In [31]:
hkls_details.head()

,Name,Year,hkls_name,hkls_link,Price,Stock,Volume
0,Lagavulin,16,Lagavulin 16 Years Single Malt Scotch Whisky,https://www.hkliquorstore.com/index.php/lagavu...,HK$888.00,True,700ml
1,Ardbeg,10,Ardbeg Ten Years Old Single Malt Whisky,https://www.hkliquorstore.com/index.php/ardbeg...,HK$648.00,True,700ml
2,Balvenie,12,The Balvenie DoubleWood 12 Years Single Malt W...,https://www.hkliquorstore.com/index.php/the-ba...,HK$688.00,True,700ml
3,Laphroaig,10,Laphroaig 10 Years Single Malt Scotch Whisky,https://www.hkliquorstore.com/index.php/laphro...,HK$688.00,True,700ml
4,Glenmorangie,10,Glenmorangie The Original 10 Years Single Malt...,https://www.hkliquorstore.com/index.php/glenmo...,HK$498.00,True,700ml


## Select whisky

In [78]:
def hkls_whisky(index):
    hkls_details = pd.read_csv('hkls_details.csv')
    name, address, price = None, None, None

    if hkls_details['Stock'][index] == True:
        name = 'Hong Kong Liquor Store (10 Stores with online shop)'
        address = 'https://www.hkliquorstore.com/locations'
        price = hkls_details['Price'][index]

    return [name,address,price]

In [92]:
# hkls_details['Stock'][12] ---> no stock 
hkls_whisky(12)

[None, None, None]

In [93]:
# hkls_details['Stock'][14] ---> have stock 
hkls_whisky(14)

['Hong Kong Liquor Store (10 Stores with online shop)',
 'https://www.hkliquorstore.com/locations',
 648.0]

In [94]:
# hkls_details['Stock'][17] ---> not selling 
hkls_whisky(17)

[None, None, None]